In [2]:
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose,BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import os
import numpy as np
from PIL import Image



# define the standalone discriminator model
def define_discriminator(in_shape=(256,256,3)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 8x8 image
    n_nodes = 512 * 32*32
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((32,32, 512)))
    # upsample to 64 * 64
    model.add(Conv2DTranspose(256, (5,5), strides=(2,2), padding='same'))
    model.add(BatchNormalization(epsilon=0.001))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    # upsample to 128 * 128
    model.add(Conv2DTranspose(128, (5,5), strides=(2,2), padding='same'))
    model.add(BatchNormalization(epsilon=0.001))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))
    # upsample 256 * 256
    model.add(Conv2DTranspose(3, (5,5), strides=(2,2), padding='same',activation='tanh'))
    return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# load and prepare mnist training images
def load_real_samples():
    dataset_input = []
    for filepath in os.listdir('new/'):
        image = Image.open('new/{}'.format(filepath))

        dataset_input.append((np.asarray(image) - 127.5)/127.5)

    X_train = np.array(dataset_input)
    return X_train

# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y

# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=2):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0])
    # save plot to file
    filename = 'output/generated_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    # save plot
    final_op  = x_fake * 127.5 +127.5
    save_plot(final_op, epoch)
    # save the generator model tile file
    filename = 'generator_final_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=400, n_batch=50):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples
            X_real, y_real = generate_real_samples(dataset, half_batch)
            # generate 'fake' examples
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # create training set for the discriminator
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            # update discriminator model weights
            d_loss, _ = d_model.train_on_batch(X, y)
            # prepare points in latent space as input for the generator
            X_gan = generate_latent_points(latent_dim, n_batch)
            # create inverted labels for the fake samples
            y_gan = ones((n_batch, 1))
            # update the generator via the discriminator's error
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            # summarize loss on this batch
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        # evaluate the model performance, sometimes
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, 1/10, d=0.643, g=0.107
>1, 2/10, d=0.370, g=0.005
>1, 3/10, d=0.359, g=0.001
>1, 4/10, d=0.354, g=0.000
>1, 5/10, d=0.349, g=0.000
>1, 6/10, d=0.346, g=0.000
>1, 7/10, d=0.343, g=0.000
>1, 8/10, d=0.340, g=0.000
>1, 9/10, d=0.337, g=0.000
>1, 10/10, d=0.334, g=0.000
>2, 1/10, d=0.330, g=0.000
>2, 2/10, d=0.326, g=0.000
>2, 3/10, d=0.320, g=0.000
>2, 4/10, d=0.312, g=0.000
>2, 5/10, d=0.302, g=0.000
>2, 6/10, d=0.291, g=0.000
>2, 7/10, d=0.277, g=0.000
>2, 8/10, d=0.261, g=0.000
>2, 9/10, d=0.243, g=0.000
>2, 10/10, d=0.223, g=0.000
>3, 1/10, d=0.202, g=0.000
>3, 2/10, d=0.180, g=0.000
>3, 3/10, d=0.158, g=0.000
>3, 4/10, d=0.136, g=0.000
>3, 5/10, d=0.116, g=0.000
>3, 6/10, d=0.098, g=0.000
>3, 7/10, d=0.082, g=0.000
>3, 8/10, d=0.068, g=0.000
>3, 9/10, d=0.057, g=0.000
>3, 10/10, d=0.048, g=0.000
>4, 1/10, d=0.040, g=0.000
>4, 2/10, d=0.034, g=0.000
>4, 3/10, d=0.029, g=0.000
>4, 4/10, d=0.025, g=0.000
>4, 5/10, d=0.021, g=0.000
>4, 6/10, d=0.019, g=0.000
>4, 7/10, d=0.017, g=0.00